In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import run_single as rs

Other shocks
========

We're going to run some other shock events, based on the list of identified shocks that I found. This will help fill out the model, and I can save them in a meaningful manner (pre, post, change) so that I don't have to keep running things. To start, I'll redo the Q opening to make sure I have all of the details.

In [2]:
conn = rs.create_connection()
units = rs.get_unit_list(conn)
query = rs.build_large_query(units, '2016-11-15', '2017-02-15', include_exit=False)  # ~150 day window
units[:5], len(units)

(['R001', 'R003', 'R004', 'R005', 'R006'], 443)

In [3]:
data = rs.resample(rs.pull_data(query, conn))
data.columns = data.columns.str.upper()
data.head()

,R001,R003,R004,R005,R006,R007,R008,R009,R013,R014,...,R464,R468,R542,R543,R544,R546,R551,R570,R571,R572
date,,,,,,,,,,,,,,,,,,,,,
2016-11-15,30496.0,1422.0,3898.0,3965.0,4902.0,2828.0,2719.0,2595.0,20096.0,63016.0,...,1319.0,2587.0,9888.0,18029.0,8646.0,19607.0,20276.0,NaN,NaN,NaN
2016-11-16,31710.0,1705.0,4223.0,4342.0,5268.0,3016.0,2898.0,2725.0,20426.0,64929.0,...,1861.0,3372.0,10394.0,18904.0,13378.0,20826.0,21341.0,NaN,NaN,NaN
2016-11-17,31618.0,1700.0,4322.0,4312.0,5362.0,3080.0,2865.0,2694.0,20651.0,66041.0,...,1912.0,3350.0,10072.0,18084.0,9425.0,20781.0,21550.0,NaN,NaN,NaN
2016-11-18,31251.0,1576.0,4131.0,4339.0,5305.0,3108.0,2918.0,2683.0,19686.0,64936.0,...,2114.0,3533.0,9281.0,16949.0,8644.0,19946.0,21060.0,NaN,NaN,NaN
2016-11-19,21318.0,908.0,2392.0,2580.0,3100.0,1695.0,1520.0,1666.0,10280.0,28683.0,...,2392.0,3823.0,0.0,6127.0,3504.0,8215.0,9833.0,NaN,NaN,NaN


In [5]:
# data.index = pd.DatetimeIndex(data.index)
data.index

DatetimeIndex(['2016-11-15', '2016-11-16', '2016-11-17', '2016-11-18',
               '2016-11-19', '2016-11-20', '2016-11-21', '2016-11-22',
               '2016-11-23', '2016-11-24', '2016-11-25', '2016-11-26',
               '2016-11-27', '2016-11-28', '2016-11-29', '2016-11-30',
               '2016-12-01', '2016-12-02', '2016-12-03', '2016-12-04',
               '2016-12-05', '2016-12-06', '2016-12-07', '2016-12-08',
               '2016-12-09', '2016-12-10', '2016-12-11', '2016-12-12',
               '2016-12-13', '2016-12-14', '2016-12-15', '2016-12-16',
               '2016-12-17', '2016-12-18', '2016-12-19', '2016-12-20',
               '2016-12-21', '2016-12-22', '2016-12-23', '2016-12-24',
               '2016-12-25', '2016-12-26', '2016-12-27', '2016-12-28',
               '2016-12-29', '2016-12-30', '2016-12-31', '2017-01-01',
               '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09',
      

Now, we can do some bayesian inference. I think there's probably a way to combine the model into getting the values for all of the stations at once, but that will have to wait for another time. For now, run the ~450 separate models and store the results.

In [6]:
from datetime import datetime

print(datetime.now())

2017-06-26 23:06:34.851911


In [7]:
data.fillna(0, inplace=True)
saved_traces = rs.prediction(data)

100%|██████████| 10000/10000 [00:02<00:00, 3818.98it/s]


100%|██████████| 10000/10000 [00:02<00:00, 3767.49it/s]


In [8]:
print(datetime.now())

2017-06-26 23:32:47.848045


In [13]:
filtered = rs.filter_traces(data.columns, saved_traces, data.index.get_loc('2017-01-01'))
len(filtered)

40

*Much* more manageable! We'll say that these stations are the only ones I care about, and the start of our data from it.

Elasticity
------------

I have two choices: I can create the values fo the ~200 stations that had changepoints near the correct day, or just for the 23 that saw changes above some threshold. I'll do the latter, since that's less calculating and the others would show marginal (at best) differences and shouldn't be taken into account.

In [39]:
changed = ['R570', 'R571', 'R572']

# units, values = list(zip(*filtered.items()))
df = pd.DataFrame(filtered, index=changed).transpose()
changed_riders = df.loc[changed].iloc[:, 0].sum()
# changed_riders
df = df / changed_riders
df

,R570,R571,R572
R015,-0.070027,-0.070027,-0.070027
R018,-0.041762,-0.041762,-0.041762
R023,0.030180,0.030180,0.030180
R025,-0.044784,-0.044784,-0.044784
R027,0.040980,0.040980,0.040980
R029,-0.067465,-0.067465,-0.067465
R035,0.021490,0.021490,0.021490
R041,-0.044143,-0.044143,-0.044143
R049,-0.036581,-0.036581,-0.036581
R050,-0.055499,-0.055499,-0.055499


In [44]:
for unit in df.index:
    if unit not in df.columns:
        df[unit] = np.NaN
df = df[sorted(df.columns)]
df.loc[changed] = (1 - df[changed].transpose()) / len(changed)
df.loc[changed, changed] = 0
# df['R179']

In [46]:
df.to_csv('processed_data/second_avenue_opens.csv')

Reload
-------

Looks like dividing by the total gained for the Q wasn't the best... Let's do the average gained by each station to divide through by, so we need to multiply those numbers by 3 in this case.

In [8]:
df = pd.read_csv('processed_data/second_avenue_opens.csv')
df.set_index(df.columns[0], inplace=True)
df.index.name = 'unit'
df[['R570', 'R571', 'R572']] = df[['R570', 'R571', 'R572']] * 3
df

,R015,R018,R023,R025,R027,R029,R035,R041,R049,R050,...,R302,R319,R322,R463,R542,R543,R544,R570,R571,R572
unit,,,,,,,,,,,,,,,,,,,,,
R015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.210081,-0.210081,-0.210081
R018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.125287,-0.125287,-0.125287
R023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.090540,0.090540,0.090540
R025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.134353,-0.134353,-0.134353
R027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122941,0.122941,0.122941
R029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.202394,-0.202394,-0.202394
R035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064471,0.064471,0.064471
R041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.132429,-0.132429,-0.132429
R049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.109744,-0.109744,-0.109744


In [9]:
df.to_csv('processed_data/second_avenue_opens.csv')